# Move Chain With Coupler
In this notebook, we implemented moving a ferromagnetic domain composed of three spins across a paramagnetic domain composed of three spins under the influence of a coupler. The dynamics remains adiabatic under sufficiently slow update of the Zeeman field and we achieved more than 90% fidelity.

In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from ising_kitaev import initialize_chain, run_adiabatic_zeeman_change, rotate_to_measurement_basis, add_measurement
from ising_kitaev import move_chain, initialize_coupler

## Input parameters

In [2]:
zeeman_ferro = 0.01  # value of on-site magnetic field for ferromagnetic domain
zeeman_para = 10  # value of on-site magnetic field for paramagnetic domain
# initial configuration of domains
initial_config = np.array([zeeman_ferro, zeeman_ferro, zeeman_ferro, zeeman_para, zeeman_para, zeeman_para]) 
# final configuration of domains
final_config = np.array([zeeman_para, zeeman_para, zeeman_para, zeeman_ferro, zeeman_ferro, zeeman_ferro]) 
qreg = QuantumRegister(7)
creg = ClassicalRegister(3)
qcirc = QuantumCircuit(qreg, creg)

## Preparing the initial state

In [3]:
initialize_chain(qcirc, qreg, initial_config, 'logical_zero')
initialize_coupler(qcirc, qreg)  # couple the chain to a coupler
qcirc.draw()
# moving ferromagenetic domain to one end
move_chain(qcirc, qreg, initial_config, final_config, 1.4, 0.25, 0.25, 2, 20, method = "both")
# moving back the ferromagnetic domain
move_chain(qcirc, qreg, final_config, initial_config, 1.4, 0.25, 0.25, 2, 20, method = "both")  
qcirc.depth()

59043

## Measurement

In [4]:
rotate_to_measurement_basis(qcirc, qreg, [0, 1, 2]) # measurement in logical basis
add_measurement(qcirc, qreg, creg, [0, 1, 2])

## Perform simulation

In [5]:
from qiskit import Aer, execute
backend = Aer.get_backend('qasm_simulator')
job = execute(qcirc, backend, shots=2000)
job.status()
result = job.result()
result.get_counts()

{'110': 1, '101': 1, '111': 3, '000': 1878, '100': 1, '001': 116}

We observe that we transfer quite efficiently the state accross the chain but the fidelity is lower than in the absence of the coupler.